<a href="https://colab.research.google.com/github/greyhound101/gan_segmentation_FE/blob/main/Use_this_end2end_federated_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [2]:
import zipfile
z=zipfile.ZipFile('/content/gdrive/My Drive/segmentation/Training_Batch1.zip') 

In [3]:
for i in z.filelist:
  i=i.filename
  if ('-0.nii' in i) or ('-1.nii' in i) or ('-2.nii' in i) or ('-3.nii' in i):
    z.extract(i,'/content/')

In [4]:
pip install medpy

     |████████████████████████████████| 153kB 14.5MB/s 
     |████████████████████████████████| 47.4MB 73kB/s 
  Created wheel for medpy: filename=MedPy-0.4.0-cp37-cp37m-linux_x86_64.whl size=754482 sha256=c22b8c23708e2db43691950dbda99478c9eb19e2542f5f60c1cc45dc9487db02
  Stored in directory: /root/.cache/pip/wheels/8c/c9/9c/2c6281c7a72b9fb1570862a4f028af7ce38405008354fbf870
Successfully built medpy


In [5]:
!git clone https://github.com/xmengli999/H-DenseUNet

Cloning into 'H-DenseUNet'...
remote: Enumerating objects: 712, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 712 (delta 16), reused 0 (delta 0), pack-reused 687
Receiving objects: 100% (712/712), 13.77 MiB | 14.85 MiB/s, done.
Resolving deltas: 100% (245/245), done.


In [6]:
cd /content/H-DenseUNet

/content/H-DenseUNet


In [7]:
pip install -r requirements.txt

     |████████████████████████████████| 81kB 7.1MB/s 
     |████████████████████████████████| 13.6MB 12.9MB/s 
     |████████████████████████████████| 153kB 49.2MB/s 
     |████████████████████████████████| 430kB 39.3MB/s 
     |████████████████████████████████| 71kB 10.1MB/s 
     |████████████████████████████████| 450kB 39.9MB/s 
     |████████████████████████████████| 583kB 45.7MB/s 
     |████████████████████████████████| 460kB 63.1MB/s 
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [8]:
cd /content/H-DenseUNet

/content/H-DenseUNet


In [9]:
import os
try:
  os.mkdir('/content/data')
  os.mkdir('/content/data/myTrainingData')
except:
  pass

In [10]:
import shutil
import glob
for path in glob.glob('/content/media/nas/01_Datasets/CT/LITS/Training Batch 1/segmentation-*'):
  try:
    shutil.move(path,'/content/data/myTrainingData/')
  except:
    pass

In [11]:
import shutil
for path in glob.glob('/content/media/nas/01_Datasets/CT/LITS/Training Batch 1/volume-*'):
  try:
    shutil.move(path,'/content/data/myTrainingData')
  except:
    pass

In [12]:
%%writefile preprocessing.py
from medpy.io import load, save
import os
import os.path
import numpy as np


def proprecessing(image_path, save_folder):

    # if not os.path.exists("/content/data/"+save_folder):
    #     os.mkdir("/content/data/"+save_folder)
    filelist = os.listdir(image_path)
    filelist = [item for item in filelist if 'volume' in item]
    for file in filelist:
        img, img_header = load(image_path+file)
        img[img < -200] = -200
        img[img > 250] = 250
        img = np.array(img, dtype='float32')
        print ("Saving image "+file)
        save(img, "/content/data/" + save_folder + file)

def generate_livertxt(image_path, save_folder):
    if not os.path.exists("/content/data/"+save_folder):
        os.mkdir("/content/data/"+save_folder)

    # Generate Livertxt
    if not os.path.exists("/content/data/"+save_folder+'LiverPixels'):
        os.mkdir("/content/data/"+save_folder+'LiverPixels')

    for i in range(0,4):
        livertumor, header = load(image_path+'segmentation-'+str(i)+'.nii')
        f = open('/content/data/' +save_folder+'/LiverPixels/liver_' + str(i) + '.txt', 'w')
        index = np.where(livertumor==1)
        x = index[0]
        y = index[1]
        z = index[2]
        np.savetxt(f, np.c_[x,y,z], fmt="%d")
	
        f.write("\n")
        f.close()

def generate_tumortxt(image_path, save_folder):
    if not os.path.exists("/content/data/"+save_folder):
        os.mkdir("/content/data/"+save_folder)

    # Generate Livertxt
    if not os.path.exists("/content/data/"+save_folder+'TumorPixels'):
        os.mkdir("/content/data/"+save_folder+'TumorPixels')

    for i in range(0,4):
        livertumor, header = load(image_path+'segmentation-'+str(i)+'.nii')
        f = open("/content/data/"+save_folder+"/TumorPixels/tumor_"+str(i)+'.txt','w')
        index = np.where(livertumor==2)

        x = index[0]
        y = index[1]
        z = index[2]

        np.savetxt(f,np.c_[x,y,z],fmt="%d")

        f.write("\n")
        f.close()

def generate_txt(image_path, save_folder):
    if not os.path.exists("/content/data/"+save_folder):
        os.mkdir("/content/data/"+save_folder)

    # Generate Livertxt
    if not os.path.exists("/content/data/"+save_folder+'LiverBox'):
        os.mkdir("/content/data/"+save_folder+'LiverBox')
    for i in range(0,4):
        values = np.loadtxt('/content/data/myTrainingDataTxt/LiverPixels/liver_' + str(i) + '.txt', delimiter=' ', usecols=[0, 1, 2])
        a = np.min(values, axis=0)
        b = np.max(values, axis=0)
        box = np.append(a,b, axis=0)
        np.savetxt('/content/data/myTrainingDataTxt/LiverBox/box_'+str(i)+'.txt', box,fmt='%d')


proprecessing(image_path='/content/data/myTrainingData/', save_folder='myTrainingData/')
# proprecessing(image_path='/content/data/TestData/', save_folder='myTestData/')
# print ("Generate liver txt ")
generate_livertxt(image_path='/content/data/myTrainingData/', save_folder='myTrainingDataTxt/')
print ("Generate tumor txt")
generate_tumortxt(image_path='/content/data/myTrainingData/', save_folder='myTrainingDataTxt/')
print ("Generate liver box ")
generate_txt(image_path='/content/data/myTrainingData/', save_folder='myTrainingDataTxt/')


Overwriting preprocessing.py


In [13]:
!python preprocessing.py 

Saving image volume-1.nii
Saving image volume-3.nii
Saving image volume-0.nii
Saving image volume-2.nii
Generate tumor txt
Generate liver box 


In [1]:
%%writefile custom_layers.py
#change
#keras 2.0.8
# import sys
# sys.path.insert(0,'Keras-2.0.8')
from keras.layers import Layer, InputSpec
try:
    from keras import initializations
except ImportError:
    from keras import initializers as initializations
import keras.backend as K

class Scale(Layer):
    '''Custom Layer for DenseNet used for BatchNormalization.
    
    Learns a set of weights and biases used for scaling the input data.
    the output consists simply in an element-wise multiplication of the input
    and a sum of a set of constants:
        out = in * gamma + beta,
    where 'gamma' and 'beta' are the weights and biases larned.
    # Arguments
        axis: integer, axis along which to normalize in mode 0. For instance,
            if your input tensor has shape (samples, channels, rows, cols),
            set axis to 1 to normalize per feature map (channels axis).
        momentum: momentum in the computation of the
            exponential average of the mean and standard deviation
            of the data, for feature-wise normalization.
        weights: Initialization weights.
            List of 2 Numpy arrays, with shapes:
            `[(input_shape,), (input_shape,)]`
        beta_init: name of initialization function for shift parameter
            (see [initializations](../initializations.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
        gamma_init: name of initialization function for scale parameter (see
            [initializations](../initializations.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
    '''
    def __init__(self, weights=None, axis=-1, momentum = 0.9, beta_init='zero', gamma_init='one', **kwargs):
        self.momentum = momentum
        self.axis = axis
        self.beta_init = initializations.get(beta_init)
        self.gamma_init = initializations.get(gamma_init)
        self.initial_weights = weights
        super(Scale, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (int(input_shape[self.axis]),)

        # Tensorflow >= 1.0.0 compatibility
        self.gamma = K.variable(self.gamma_init(shape), name='{}_gamma'.format(self.name))
        self.beta = K.variable(self.beta_init(shape), name='{}_beta'.format(self.name))
        #self.gamma = self.gamma_init(shape, name='{}_gamma'.format(self.name))
        #self.beta = self.beta_init(shape, name='{}_beta'.format(self.name))
        self.trainable_weights_ = [self.gamma, self.beta]

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def call(self, x, mask=None):
        input_shape = self.input_spec[0].shape
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        out = K.reshape(self.gamma, broadcast_shape) * x + K.reshape(self.beta, broadcast_shape)
        return out

    def get_config(self):
        config = {"momentum": self.momentum, "axis": self.axis}
        base_config = super(Scale, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

Overwriting custom_layers.py


In [2]:
%%writefile hybridnet.py
from keras.models import Model
from keras.layers import Input, ZeroPadding2D, concatenate, Lambda, ZeroPadding3D, add
from keras.layers.core import Dropout, Activation
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv3D, UpSampling3D, AveragePooling3D
from keras.layers.pooling import AveragePooling2D, MaxPooling2D, MaxPooling3D
from keras.layers.normalization import BatchNormalization
from custom_layers import Scale
import tensorflow as tf


def conv_block3d(x, stage, branch, nb_filter, dropout_rate=None, weight_decay=1e-4):
    '''Apply BatchNorm, Relu, bottleneck 1x1 Conv3D, 3x3 Conv3D, and option dropout
        # Arguments
            x: input tensor
            stage: index for dense block
            branch: layer index within each dense block
            nb_filter: number of filters
            dropout_rate: dropout rate
            weight_decay: weight decay factor
    '''
    eps = 1.1e-5
    conv_name_base = '3dconv' + str(stage) + '_' + str(branch)
    relu_name_base = '3drelu' + str(stage) + '_' + str(branch)

    # 1x1 Convolution (Bottleneck layer)
    inter_channel = nb_filter * 4
    x = BatchNormalization(epsilon=eps, axis=4, name=conv_name_base+'_x1_bn', momentum=1.0, trainable=False)(x, training=False)
    x = Scale(axis=4, name=conv_name_base+'_x1_scale')(x)
    x = Activation('relu', name=relu_name_base+'_x1')(x)
    x = Conv3D(inter_channel, (1, 1, 1), name=conv_name_base+'_x1', use_bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    # 3x3 Convolution
    x = BatchNormalization(epsilon=eps, axis=4, name=conv_name_base+'_x2_bn', momentum=1.0, trainable=False)(x, training=False)
    x = Scale(axis=4, name=conv_name_base+'_x2_scale')(x)
    x = Activation('relu', name=relu_name_base+'_x2')(x)
    x = ZeroPadding3D((1, 1, 1), name=conv_name_base+'_x2_zeropadding')(x)
    x = Conv3D(nb_filter, (3, 3, 3), name=conv_name_base+'_x2', use_bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    return x
def dense_block3d(x, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1e-4, grow_nb_filters=True):
    ''' Build a dense_block where the output of each conv_block is fed to subsequent ones
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_layers: the number of layers of conv_block to append to the model.
            nb_filter: number of filters
            growth_rate: growth rate
            dropout_rate: dropout rate
            weight_decay: weight decay factor
            grow_nb_filters: flag to decide to allow number of filters to grow
    '''

    eps = 1.1e-5
    concat_feat = x

    for i in range(nb_layers):
        branch = i+1
        x = conv_block3d(concat_feat, stage, branch, growth_rate, dropout_rate, weight_decay)
        concat_feat = concatenate([concat_feat, x], axis=4, name='3dconcat_'+str(stage)+'_'+str(branch))

        if grow_nb_filters:
            nb_filter += growth_rate

    return concat_feat, nb_filter
def transition_block3d(x, stage, nb_filter, compression=1.0, dropout_rate=None, weight_decay=1E-4):
    ''' Apply BatchNorm, 1x1 Convolution, averagePooling, optional compression, dropout
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_filter: number of filters
            compression: calculated as 1 - reduction. Reduces the number of feature maps in the transition block.
            dropout_rate: dropout rate
            weight_decay: weight decay factor
    '''

    eps = 1.1e-5
    conv_name_base = '3dconv' + str(stage) + '_blk'
    relu_name_base = '3drelu' + str(stage) + '_blk'
    pool_name_base = '3dpool' + str(stage)

    x = BatchNormalization(epsilon=eps, axis=4, name=conv_name_base+'_bn', momentum=1.0)(x, training=False)
    x = Scale(axis=4, name=conv_name_base+'_scale')(x)
    x = Activation('relu', name=relu_name_base)(x)
    x = Conv3D(int(nb_filter * compression), (1, 1, 1), name=conv_name_base, use_bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    x = AveragePooling3D((2, 2, 1), strides=(2, 2, 1), name=pool_name_base)(x)

    return x
def DenseNet3D(img_input, nb_dense_block=4, growth_rate=32, nb_filter=96, reduction=0.0, dropout_rate=0.0, weight_decay=1e-4, classes=1000, weights_path=None):
    '''Instantiate the DenseNet 161 architecture,
        # Arguments
            nb_dense_block: number of dense blocks to add to end
            growth_rate: number of filters to add per dense block
            nb_filter: initial number of filters
            reduction: reduction factor of transition blocks.
            dropout_rate: dropout rate
            weight_decay: weight decay factor
            classes: optional number of classes to classify images
            weights_path: path to pre-trained weights
        # Returns
            A Keras model instance.
    '''
    eps = 1.1e-5

    # compute compression factor
    compression = 1.0 - reduction

    # From architecture for ImageNet (Table 1 in the paper)
    nb_filter = 96
    nb_layers = [3, 4, 12, 8]  # For DenseNet-161
    box = []
    # Initial convolution
    x = ZeroPadding3D((3, 3, 3), name='3dconv1_zeropadding')(img_input)
    x = Conv3D(nb_filter, (7, 7, 7), strides=(2, 2, 2), name='3dconv1', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=4, name='3dconv1_bn')(x)
    x = Scale(axis=4, name='3dconv1_scale')(x)
    x = Activation('relu', name='3drelu1')(x)
    box.append(x)
    x = ZeroPadding3D((1, 1, 1), name='3dpool1_zeropadding')(x)
    x = MaxPooling3D((3, 3, 3), strides=(2, 2, 2), name='3dpool1')(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        stage = block_idx + 2
        x, nb_filter = dense_block3d(x, stage, nb_layers[block_idx], nb_filter, growth_rate, dropout_rate=dropout_rate,
                                   weight_decay=weight_decay)
        box.append(x)
        # Add transition_block
        x = transition_block3d(x, stage, nb_filter, compression=compression, dropout_rate=dropout_rate,
                             weight_decay=weight_decay)
        nb_filter = int(nb_filter * compression)

    final_stage = stage + 1
    x, nb_filter = dense_block3d(x, final_stage, nb_layers[-1], nb_filter, growth_rate, dropout_rate=dropout_rate,
                               weight_decay=weight_decay)

    x = BatchNormalization(epsilon=eps, axis=4, name='3dconv' + str(final_stage) + '_blk_bn')(x)
    x = Scale(axis=4, name='3dconv' + str(final_stage) + '_blk_scale')(x)
    x = Activation('relu', name='3drelu' + str(final_stage) + '_blk')(x)
    box.append(x)

    up0 = UpSampling3D(size=(2, 2, 1))(x)
    conv_up0 = Conv3D(504, (3, 3, 3), padding="same", name="3dconv_up0")(up0)
    bn_up0 = BatchNormalization(name="3dbn_up0")(conv_up0)
    ac_up0 = Activation('relu', name='3dac_up0')(bn_up0)

    up1 = UpSampling3D(size=(2, 2, 1))(ac_up0)
    conv_up1 = Conv3D(224, (3, 3, 3), padding="same", name="3dconv_up1")(up1)
    bn_up1 = BatchNormalization(name="3dbn_up1")(conv_up1)
    ac_up1 = Activation('relu', name='3dac_up1')(bn_up1)

    up2 = UpSampling3D(size=(2, 2, 1))(ac_up1)
    conv_up2 = Conv3D(192, (3, 3, 3), padding="same", name="3dconv_up2")(up2)
    bn_up2 = BatchNormalization(name="3dbn_up2")(conv_up2)
    ac_up2 = Activation('relu', name='3dac_up2')(bn_up2)

    up3 = UpSampling3D(size=(2, 2, 2))(ac_up2)
    conv_up3 = Conv3D(96, (3, 3, 3), padding="same", name="3dconv_up3")(up3)
    bn_up3 = BatchNormalization(name="3dbn_up3")(conv_up3)
    ac_up3 = Activation('relu', name='3dac_up3')(bn_up3)

    up4 = UpSampling3D(size=(2, 2, 2))(ac_up3)
    conv_up4 = Conv3D(64, (3, 3, 3), padding="same", name="3dconv_up4")(up4)
    bn_up4 = BatchNormalization(name="3dbn_up4")(conv_up4)
    ac_up4 = Activation('relu', name='3dac_up4')(bn_up4)

    x = Conv3D(3, (1, 1, 1), padding="same", name='3dclassifer')(ac_up4)

    return ac_up4, x



def DenseUNet(img_input, nb_dense_block=4, growth_rate=48, nb_filter=96, reduction=0.0, dropout_rate=0.0, weight_decay=1e-4, classes=1000, weights_path=None):
    '''Instantiate the DenseNet 161 architecture,
        # Arguments
            nb_dense_block: number of dense blocks to add to end
            growth_rate: number of filters to add per dense block
            nb_filter: initial number of filters
            reduction: reduction factor of transition blocks.
            dropout_rate: dropout rate
            weight_decay: weight decay factor
            classes: optional number of classes to classify images
            weights_path: path to pre-trained weights
        # Returns
            A Keras model instance.
    '''
    eps = 1.1e-5
    # compute compression factor
    compression = 1.0 - reduction

    # Handle Dimension Ordering for different backends
    global concat_axis
    concat_axis = 3

    # From architecture for ImageNet (Table 1 in the paper)
    nb_filter = 96
    nb_layers = [6,12,36,24] # For DenseNet-161
    box = []
    # Initial convolution
    x = ZeroPadding2D((3, 3), name='conv1_zeropadding')(img_input)
    x = Conv2D(nb_filter, (7, 7), strides=(2, 2), name='conv1', use_bias=False, trainable=True)(x)
    x = BatchNormalization(epsilon=eps, axis=concat_axis, momentum = 1, name='conv1_bn', trainable=False)(x, training=False)
    x = Scale(axis=concat_axis, name='conv1_scale')(x)
    x = Activation('relu', name='relu1')(x)
    box.append(x)
    x = ZeroPadding2D((1, 1), name='pool1_zeropadding')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        stage = block_idx+2
        x, nb_filter = dense_block(x, stage, nb_layers[block_idx], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)
        box.append(x)
        # Add transition_block
        x = transition_block(x, stage, nb_filter, compression=compression, dropout_rate=dropout_rate, weight_decay=weight_decay)
        nb_filter = int(nb_filter * compression)

    final_stage = stage + 1
    x, nb_filter = dense_block(x, final_stage, nb_layers[-1], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)

    x = BatchNormalization(epsilon=eps, axis=concat_axis, momentum = 1, name='conv'+str(final_stage)+'_blk_bn', trainable=False)(x, training=False)
    x = Scale(axis=concat_axis, name='conv'+str(final_stage)+'_blk_scale')(x)
    x = Activation('relu', name='relu'+str(final_stage)+'_blk')(x)
    box.append(x)

    up0 = UpSampling2D(size=(2,2))(x)
    conv_up0 = Conv2D(768, (3, 3), padding="same", name = "conv_up0", trainable=True)(up0)
    bn_up0 = BatchNormalization(name = "bn_up0", momentum = 1, trainable=False)(conv_up0, training=False)
    ac_up0 = Activation('relu', name='ac_up0')(bn_up0)

    up1 = UpSampling2D(size=(2,2))(ac_up0)
    conv_up1 = Conv2D(384, (3, 3), padding="same", name = "conv_up1", trainable=True)(up1)
    bn_up1 = BatchNormalization(name = "bn_up1", momentum = 1, trainable=False)(conv_up1, training=False)
    ac_up1 = Activation('relu', name='ac_up1')(bn_up1)

    up2 = UpSampling2D(size=(2,2))(ac_up1)
    conv_up2 = Conv2D(96, (3, 3), padding="same", name = "conv_up2", trainable=True)(up2)
    bn_up2 = BatchNormalization(name = "bn_up2", momentum = 1, trainable=False)(conv_up2, training=False)
    ac_up2 = Activation('relu', name='ac_up2')(bn_up2)

    up3 = UpSampling2D(size=(2,2))(ac_up2)
    conv_up3 = Conv2D(96, (3, 3), padding="same", name = "conv_up3", trainable=True)(up3)
    bn_up3 = BatchNormalization(name = "bn_up3", momentum = 1, trainable=False)(conv_up3, training=False)
    ac_up3 = Activation('relu', name='ac_up3')(bn_up3)

    up4 = UpSampling2D(size=(2, 2))(ac_up3)
    conv_up4 = Conv2D(64, (3, 3), padding="same", name="conv_up4", trainable=True)(up4)
    bn_up4 = BatchNormalization(name="bn_up4", momentum = 1, trainable=False)(conv_up4, training=False)
    ac_up4 = Activation('relu', name='ac_up4')(bn_up4)

    x = Conv2D(3, (1,1), padding="same", name='dense167classifer', trainable=True)(ac_up4)

    return ac_up4, x

def conv_block(x, stage, branch, nb_filter, dropout_rate=None, weight_decay=1e-4):
    '''Apply BatchNorm, Relu, bottleneck 1x1 Conv2D, 3x3 Conv2D, and option dropout
        # Arguments
            x: input tensor
            stage: index for dense block
            branch: layer index within each dense block
            nb_filter: number of filters
            dropout_rate: dropout rate
            weight_decay: weight decay factor
    '''
    eps = 1.1e-5
    conv_name_base = 'conv' + str(stage) + '_' + str(branch)
    relu_name_base = 'relu' + str(stage) + '_' + str(branch)

    # 1x1 Convolution (Bottleneck layer)
    inter_channel = nb_filter * 4
    x = BatchNormalization(epsilon=eps, axis=concat_axis, momentum = 1, name=conv_name_base+'_x1_bn', trainable=False)(x, training=False)
    x = Scale(axis=concat_axis, name=conv_name_base+'_x1_scale')(x)
    x = Activation('relu', name=relu_name_base+'_x1')(x)
    x = Conv2D(inter_channel, (1, 1), name=conv_name_base+'_x1', use_bias=False, trainable=True)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    # 3x3 Convolution
    x = BatchNormalization(epsilon=eps, axis=concat_axis, momentum = 1, name=conv_name_base+'_x2_bn', trainable=False)(x, training=False)
    x = Scale(axis=concat_axis, name=conv_name_base+'_x2_scale')(x)
    x = Activation('relu', name=relu_name_base+'_x2')(x)
    x = ZeroPadding2D((1, 1), name=conv_name_base+'_x2_zeropadding')(x)
    x = Conv2D(nb_filter, (3, 3), name=conv_name_base+'_x2', use_bias=False, trainable=True)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    return x


def transition_block(x, stage, nb_filter, compression=1.0, dropout_rate=None, weight_decay=1E-4):
    ''' Apply BatchNorm, 1x1 Convolution, averagePooling, optional compression, dropout
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_filter: number of filters
            compression: calculated as 1 - reduction. Reduces the number of feature maps in the transition block.
            dropout_rate: dropout rate
            weight_decay: weight decay factor
    '''

    eps = 1.1e-5
    conv_name_base = 'conv' + str(stage) + '_blk'
    relu_name_base = 'relu' + str(stage) + '_blk'
    pool_name_base = 'pool' + str(stage)

    x = BatchNormalization(epsilon=eps, axis=concat_axis, momentum = 1, name=conv_name_base+'_bn', trainable=False)(x, training=False)
    x = Scale(axis=concat_axis, name=conv_name_base+'_scale')(x)
    x = Activation('relu', name=relu_name_base)(x)
    x = Conv2D(int(nb_filter * compression), (1, 1), name=conv_name_base, use_bias=False, trainable=True)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    x = AveragePooling2D((2, 2), strides=(2, 2), name=pool_name_base)(x)

    return x


def dense_block(x, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1e-4, grow_nb_filters=True):
    ''' Build a dense_block where the output of each conv_block is fed to subsequent ones
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_layers: the number of layers of conv_block to append to the model.
            nb_filter: number of filters
            growth_rate: growth rate
            dropout_rate: dropout rate
            weight_decay: weight decay factor
            grow_nb_filters: flag to decide to allow number of filters to grow
    '''

    eps = 1.1e-5
    concat_feat = x

    for i in range(nb_layers):
        branch = i+1
        x = conv_block(concat_feat, stage, branch, growth_rate, dropout_rate, weight_decay)
        concat_feat = concatenate([concat_feat, x], axis=concat_axis, name='concat_'+str(stage)+'_'+str(branch))

        if grow_nb_filters:
            nb_filter += growth_rate

    return concat_feat, nb_filter
def slice(x, h1, h2):
    """ Define a tensor slice function
    """
    return x[:, :, :, h1:h2,:]
def slice2d(x, h1, h2):

    tmp = x[h1:h2,:,:,:]
    tmp = tf.transpose(tmp, perm=[1, 2, 0, 3])
    tmp = tf.expand_dims(tmp, 0)
    return tmp

def slice_last(x):

    x = x[:,:,:,:,0]
    return x
def trans(x):

    x = tf.transpose(x, perm=[0,3,1,2,4])
    return x
def trans_back(x):

    x = tf.transpose(x, perm=[0,2,3,1,4])

    return x
def dense_rnn_net(args):

    #  ************************3d volume input******************************************************************
    img_input = Input(batch_shape=(args.b, args.input_size, args.input_size, args.input_cols, 1), name='volumetric_data')

    #  ************************(batch*d3cols)*2dvolume--2D DenseNet branch**************************************
    input2d = Lambda(slice, arguments={'h1': 0, 'h2': 2})(img_input)
    single = Lambda(slice, arguments={'h1':0, 'h2':1})(img_input)
    input2d = concatenate([single, input2d], axis=3)
    for i in range(args.input_cols - 2):
        input2d_tmp = Lambda(slice, arguments={'h1': i, 'h2': i + 3})(img_input)
        input2d = concatenate([input2d, input2d_tmp], axis=0)
        if i == args.input_cols - 3:
            final1 = Lambda(slice, arguments={'h1': args.input_cols-2, 'h2': args.input_cols})(img_input)
            final2 = Lambda(slice, arguments={'h1': args.input_cols-1, 'h2': args.input_cols})(img_input)
            final = concatenate([final1, final2], axis=3)
            input2d = concatenate([input2d, final], axis=0)
    input2d = Lambda(slice_last)(input2d)

    #  ******************************stack to 3D volumes *******************************************************
    feature2d, classifer2d = DenseUNet(input2d, reduction=0.5)
    res2d = Lambda(slice2d, arguments={'h1': 0, 'h2': 1})(classifer2d)
    fea2d = Lambda(slice2d, arguments={'h1':0, 'h2':1})(feature2d)
    for j in range(args.input_cols - 1):
        score = Lambda(slice2d, arguments={'h1': j + 1, 'h2': j + 2})(classifer2d)
        fea2d_slice = Lambda(slice2d, arguments={'h1': j + 1, 'h2': j + 2})(feature2d)
        res2d = concatenate([res2d, score], axis=3)
        fea2d = concatenate([fea2d, fea2d_slice], axis=3)

    #  *************************** 3d DenseNet on 3D volume (concate with feature map )*********************************
    res2d_input = Lambda(lambda x: x * 250)(res2d)
    input3d_ori = Lambda(slice, arguments={'h1': 0, 'h2': args.input_cols})(img_input)
    input3d = concatenate([input3d_ori, res2d_input], axis=4)
    feature3d, classifer3d = DenseNet3D(input3d, reduction=0.5)

    final = add([feature3d, fea2d])
    final_conv = Conv3D(64, (3, 3, 3), padding="same", name='fianl_conv')(final)
    final_conv = Dropout(rate=0.3)(final_conv)
    final_bn = BatchNormalization(name="final_bn")(final_conv)
    final_ac = Activation('relu', name='final_ac')(final_bn)
    classifer = Conv3D(3, (1, 1, 1), padding="same", name='2d3dclassifer')(final_ac)

    model = Model( inputs = img_input,outputs = classifer, name='auto3d_residual_conv')

    return model


def dilated_resnet(args):
    inputs = Input(batch_shape = (args.b, args.input_size, args.input_size, args.input_cols, 1))
    conv1 = Conv3D(64, (3, 3, 3), padding = "same",kernel_initializer="normal")(inputs)
    bn0 = BatchNormalization()(conv1)
    ac0 = Activation('relu')(bn0)
    pool1 = MaxPooling3D(pool_size=(2, 2, 1))(ac0)

    #  resudial block
    conv2 = Conv3D(128, (3, 3, 3), padding="same", kernel_initializer="normal")(pool1)
    bn1 = BatchNormalization()(conv2)
    ac1 = Activation('relu')(bn1)
    conv3 = Conv3D(128, (3, 3, 3), padding="same", kernel_initializer="normal")(ac1)
    bn2 = BatchNormalization()(conv3)
    pad1 = Conv3D(128, (1, 1, 1), padding="same", kernel_initializer="normal")(pool1)
    BN1 = BatchNormalization()(pad1)
    sumb1 = add([BN1, bn2])
    res1  = Activation('relu')(sumb1)

    pool2 = MaxPooling3D(pool_size=(2, 2, 1))(res1)

    #  resudial block
    conv4 = Conv3D(256, (3, 3, 3), padding="same", kernel_initializer="normal")(pool2)
    bn3 = BatchNormalization()(conv4)
    ac2 = Activation('relu')(bn3)
    conv5 = Conv3D(256, (3, 3, 3), padding="same", kernel_initializer="normal")(ac2)
    bn4 = BatchNormalization()(conv5)
    pad2 = Conv3D(256, (1, 1, 1), padding="same", kernel_initializer="normal")(pool2)
    BN2 = BatchNormalization()(pad2)
    sumb2 = add([BN2, bn4])
    res2  = Activation('relu')(sumb2)


    pool3 = MaxPooling3D(pool_size=(2, 2, 1))(res2)

    #  resudial block
    conv6 = Conv3D(512, (3, 3, 3), padding="same", kernel_initializer="normal")(pool3)
    bn5 = BatchNormalization()(conv6)
    ac3 = Activation('relu')(bn5)
    conv7 = Conv3D(512, (3, 3, 3), padding="same", kernel_initializer="normal")(ac3)
    bn6 = BatchNormalization()(conv7)
    pad3 = Conv3D(512, (1, 1, 1), padding="same", kernel_initializer="normal")(pool3)
    BN3 = BatchNormalization()(pad3)
    sumb3 = add([BN3, bn6])
    res3  = Activation('relu')(sumb3)

    #  resudial deliated block
    del1 = Conv3D(512, (3, 3, 3), padding="same", dilation_rate=(2, 2, 2), kernel_initializer="normal")(res3)
    delbn1 = BatchNormalization()(del1)
    delac1 = Activation('relu')(delbn1)
    del2 = Conv3D(512, (3, 3, 3), padding="same", dilation_rate=(2, 2, 2), kernel_initializer="normal")(delac1)
    delbn2 = BatchNormalization()(del2)
    deladd1 = add([res3, delbn2])
    delres  = Activation('relu')(deladd1)

    pool4 = MaxPooling3D(pool_size=(2, 2, 1))(delres)

    #  resudial block
    conv6_4 = Conv3D(512, (3, 3, 3), padding="same", kernel_initializer="normal")(pool4)
    bn5_4 = BatchNormalization()(conv6_4)
    ac3_4 = Activation('relu')(bn5_4)
    conv7_4 = Conv3D(512, (3, 3, 3), padding="same", kernel_initializer="normal")(ac3_4)
    bn6_4 = BatchNormalization()(conv7_4)
    pad3_4 = Conv3D(512, (1, 1, 1), padding="same", kernel_initializer="normal")(pool4)
    BN3_4 = BatchNormalization()(pad3_4)
    sumb3_4 = add([BN3_4, bn6_4])
    res3_4  = Activation('relu')(sumb3_4)

    #  resudial deliated block
    del3 = Conv3D(512, (3, 3, 3), padding="same", dilation_rate=(2, 2, 2), kernel_initializer="normal")(res3_4)
    delbn3 = BatchNormalization()(del3)
    delac3 = Activation('relu')(delbn3)
    del4 = Conv3D(512, (3, 3, 3), padding="same", dilation_rate=(2, 2, 2), kernel_initializer="normal")(delac3)
    delbn4 = BatchNormalization()(del4)
    deladd2 = add([res3_4, delbn4])
    delres2  = Activation('relu')(deladd2)


    up0 = UpSampling3D(size=(2,2,1))(delres2)
    pad4 = Conv3D(512, (1, 1, 1), padding="same", kernel_initializer="normal")(delres)
    BN4 = BatchNormalization()(pad4)
    sumb4 = add([BN4, up0])

    #  resudial block
    conv8_1 = Conv3D(512, (3, 3, 3), padding="same", kernel_initializer="normal")(sumb4)
    bn7_1 = BatchNormalization()(conv8_1)
    ac4_1 = Activation('relu')(bn7_1)
    conv9_1 = Conv3D(512, (3, 3, 3), padding="same", kernel_initializer="normal")(ac4_1)
    bn8_1 = BatchNormalization()(conv9_1)
    pad5_1 = Conv3D(512, (1, 1, 1), padding="same", kernel_initializer="normal")(sumb4)
    BN5_1 = BatchNormalization()(pad5_1)
    sumb5_1 = add([BN5_1, bn8_1])
    res4_1  = Activation('relu')(sumb5_1)

    #  resudial deliated block
    del5 = Conv3D(512, (3, 3, 3), padding="same", dilation_rate=(2, 2, 2), kernel_initializer="normal")(res4_1)
    delbn5 = BatchNormalization()(del5)
    delac5 = Activation('relu')(delbn5)
    del6 = Conv3D(512, (3, 3, 3), padding="same", dilation_rate=(2, 2, 2), kernel_initializer="normal")(delac5)
    delbn6 = BatchNormalization()(del6)
    deladd3 = add([res4_1, delbn6])
    delres3  = Activation('relu')(deladd3)

    up0_1 = UpSampling3D(size=(2,2,1))(delres3)
    pad4_1 = Conv3D(512, (1, 1, 1), padding="same", kernel_initializer="normal")(res2)
    BN4_1 = BatchNormalization()(pad4_1)
    sumb4_1 = add([BN4_1, up0_1])

    #  resudial block
    conv8 = Conv3D(256, (3, 3, 3), padding="same", kernel_initializer="normal")(sumb4_1)
    bn7 = BatchNormalization()(conv8)
    ac4 = Activation('relu')(bn7)
    conv9 = Conv3D(256, (3, 3, 3), padding="same", kernel_initializer="normal")(ac4)
    bn8 = BatchNormalization()(conv9)
    pad5 = Conv3D(256, (1, 1, 1), padding="same", kernel_initializer="normal")(sumb4_1)
    BN5 = BatchNormalization()(pad5)
    sumb5 = add([BN5, bn8])
    res4  = Activation('relu')(sumb5)

    up1 = UpSampling3D(size=(2, 2, 1))(res4)
    pad6 = Conv3D(256, (1, 1, 1), padding="same", kernel_initializer="normal")(res1)
    BN6 = BatchNormalization()(pad6)
    sumb6 = add([BN6, up1])

    #  resudial block
    conv10 = Conv3D(128, (3, 3, 3), padding="same", kernel_initializer="normal")(sumb6)
    bn9 = BatchNormalization()(conv10)
    ac5 = Activation('relu')(bn9)
    conv11 = Conv3D(128, (3, 3, 3), padding="same", kernel_initializer="normal")(ac5)
    bn10 = BatchNormalization()(conv11)
    pad7 = Conv3D(128, (1, 1, 1), padding="same", kernel_initializer="normal")(sumb6)
    BN7 = BatchNormalization()(pad7)
    sumb7 = add([BN7, bn10])
    res5  = Activation('relu')(sumb7)

    up2 = UpSampling3D(size=(2, 2, 1))(res5)
    pad8 = Conv3D(128, (1, 1, 1), padding="same", kernel_initializer="normal")(ac0)
    BN8 = BatchNormalization()(pad8)
    sumb8 = add([BN8, up2])

    #  resudial block
    conv12 = Conv3D(64, (3, 3, 3), padding="same", kernel_initializer="normal")(sumb8)
    bn11= BatchNormalization()(conv12)
    ac6 = Activation('relu')(bn11)
    conv13 = Conv3D(64, (3, 3, 3), padding="same", kernel_initializer="normal")(ac6)
    bn12 = BatchNormalization()(conv13)
    pad9 = Conv3D(64, (1, 1, 1), padding="same", kernel_initializer="normal")(sumb8)
    BN9 = BatchNormalization()(pad9)
    sumb9 = add([BN9, bn12])
    res6 = Activation('relu')(sumb9)

    output3 = Conv3D(2, (1, 1, 1), padding="same", kernel_initializer="normal")(res6)

    # print (output3)


    model = Model(inputs=[inputs], outputs=[output3])



    return model

Overwriting hybridnet.py


In [3]:
%%writefile denseunet3d.py
"""Test ImageNet pretrained DenseNet"""
from __future__ import print_function
#change
#keras 2.0.8
# import sys 
# sys.path.insert(0,'Keras-2.0.8')
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, ZeroPadding2D, concatenate, Lambda, ZeroPadding3D, add
from keras.layers.core import Dropout, Activation
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv3D, UpSampling3D, AveragePooling3D
from keras.layers.pooling import AveragePooling2D, MaxPooling2D, MaxPooling3D
from keras.layers.normalization import BatchNormalization
from custom_layers import Scale
import keras.backend as K
import os


def conv_block3d(x, stage, branch, nb_filter, dropout_rate=None, weight_decay=1e-4):
    '''Apply BatchNorm, Relu, bottleneck 1x1 Conv3D, 3x3 Conv3D, and option dropout
        # Arguments
            x: input tensor 
            stage: index for dense block
            branch: layer index within each dense block
            nb_filter: number of filters
            dropout_rate: dropout rate
            weight_decay: weight decay factor
    '''
    eps = 1.1e-5
    conv_name_base = '3dconv' + str(stage) + '_' + str(branch)
    relu_name_base = '3drelu' + str(stage) + '_' + str(branch)

    # 1x1 Convolution (Bottleneck layer)
    inter_channel = nb_filter * 4
    x = BatchNormalization(epsilon=eps, axis=4, name=conv_name_base+'_x1_bn')(x)
    x = Scale(axis=4, name=conv_name_base+'_x1_scale')(x)
    x = Activation('relu', name=relu_name_base+'_x1')(x)
    x = Conv3D(inter_channel, (1, 1, 1), name=conv_name_base+'_x1', use_bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    # 3x3 Convolution
    x = BatchNormalization(epsilon=eps, axis=4, name=conv_name_base+'_x2_bn')(x)
    x = Scale(axis=4, name=conv_name_base+'_x2_scale')(x)
    x = Activation('relu', name=relu_name_base+'_x2')(x)
    x = ZeroPadding3D((1, 1, 1), name=conv_name_base+'_x2_zeropadding')(x)
    x = Conv3D(nb_filter, (3, 3, 3), name=conv_name_base+'_x2', use_bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    return x
def dense_block3d(x, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1e-4, grow_nb_filters=True):
    ''' Build a dense_block where the output of each conv_block is fed to subsequent ones
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_layers: the number of layers of conv_block to append to the model.
            nb_filter: number of filters
            growth_rate: growth rate
            dropout_rate: dropout rate
            weight_decay: weight decay factor
            grow_nb_filters: flag to decide to allow number of filters to grow
    '''

    eps = 1.1e-5
    concat_feat = x

    for i in range(nb_layers):
        branch = i+1
        x = conv_block3d(concat_feat, stage, branch, growth_rate, dropout_rate, weight_decay)
        concat_feat = concatenate([concat_feat, x], axis=4, name='3dconcat_'+str(stage)+'_'+str(branch))

        if grow_nb_filters:
            nb_filter += growth_rate

    return concat_feat, nb_filter
def transition_block3d(x, stage, nb_filter, compression=1.0, dropout_rate=None, weight_decay=1E-4):
    ''' Apply BatchNorm, 1x1 Convolution, averagePooling, optional compression, dropout 
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_filter: number of filters
            compression: calculated as 1 - reduction. Reduces the number of feature maps in the transition block.
            dropout_rate: dropout rate
            weight_decay: weight decay factor
    '''

    eps = 1.1e-5
    conv_name_base = '3dconv' + str(stage) + '_blk'
    relu_name_base = '3drelu' + str(stage) + '_blk'
    pool_name_base = '3dpool' + str(stage)

    x = BatchNormalization(epsilon=eps, axis=4, name=conv_name_base+'_bn')(x)
    x = Scale(axis=4, name=conv_name_base+'_scale')(x)
    x = Activation('relu', name=relu_name_base)(x)
    x = Conv3D(int(nb_filter * compression), (1, 1, 1), name=conv_name_base, use_bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    x = AveragePooling3D((2, 2, 1), strides=(2, 2, 1), name=pool_name_base)(x)

    return x
def DenseNet3D(img_input, nb_dense_block=4, growth_rate=32, nb_filter=96, reduction=0.0, dropout_rate=0.0, weight_decay=1e-4, classes=1000, weights_path=None):
    '''Instantiate the DenseNet 161 architecture,
        # Arguments
            nb_dense_block: number of dense blocks to add to end
            growth_rate: number of filters to add per dense block
            nb_filter: initial number of filters
            reduction: reduction factor of transition blocks.
            dropout_rate: dropout rate
            weight_decay: weight decay factor
            classes: optional number of classes to classify images
            weights_path: path to pre-trained weights
        # Returns
            A Keras model instance.
    '''
    eps = 1.1e-5

    # compute compression factor
    compression = 1.0 - reduction

    # From architecture for ImageNet (Table 1 in the paper)
    nb_filter = 96
    nb_layers = [3, 4, 12, 8]  # For DenseNet-161
    box = []
    # Initial convolution
    x = ZeroPadding3D((3, 3, 3), name='3dconv1_zeropadding')(img_input)
    x = Conv3D(nb_filter, (7, 7, 7), strides=(2, 2, 2), name='3dconv1', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=4, name='3dconv1_bn')(x)
    x = Scale(axis=4, name='3dconv1_scale')(x)
    x = Activation('relu', name='3drelu1')(x)
    box.append(x)
    x = ZeroPadding3D((1, 1, 1), name='3dpool1_zeropadding')(x)
    x = MaxPooling3D((3, 3, 3), strides=(2, 2, 2), name='3dpool1')(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        stage = block_idx + 2
        x, nb_filter = dense_block3d(x, stage, nb_layers[block_idx], nb_filter, growth_rate, dropout_rate=dropout_rate,
                                   weight_decay=weight_decay)
        box.append(x)
        # Add transition_block
        x = transition_block3d(x, stage, nb_filter, compression=compression, dropout_rate=dropout_rate,
                             weight_decay=weight_decay)
        nb_filter = int(nb_filter * compression)

    final_stage = stage + 1
    x, nb_filter = dense_block3d(x, final_stage, nb_layers[-1], nb_filter, growth_rate, dropout_rate=dropout_rate,
                               weight_decay=weight_decay)

    x = BatchNormalization(epsilon=eps, axis=4, name='3dconv' + str(final_stage) + '_blk_bn')(x)
    x = Scale(axis=4, name='3dconv' + str(final_stage) + '_blk_scale')(x)
    x = Activation('relu', name='3drelu' + str(final_stage) + '_blk')(x)
    box.append(x)
    # print (box)
    up0 = UpSampling3D(size=(2, 2, 1))(x)
    # line0 = Conv3D(504, (1, 1, 1), padding="same", name="3dline0")(box[3])
    # up0_sum = add([line0, up0])
    conv_up0 = Conv3D(504, (3, 3, 3), padding="same", name="3dconv_up0")(up0)
    bn_up0 = BatchNormalization(name="3dbn_up0")(conv_up0)
    ac_up0 = Activation('relu', name='3dac_up0')(bn_up0)

    up1 = UpSampling3D(size=(2, 2, 1))(ac_up0)
    # up1_sum = add([box[2], up1])
    conv_up1 = Conv3D(224, (3, 3, 3), padding="same", name="3dconv_up1")(up1)
    bn_up1 = BatchNormalization(name="3dbn_up1")(conv_up1)
    ac_up1 = Activation('relu', name='3dac_up1')(bn_up1)

    up2 = UpSampling3D(size=(2, 2, 1))(ac_up1)
    # up2_sum = add([box[1], up2])
    conv_up2 = Conv3D(192, (3, 3, 3), padding="same", name="3dconv_up2")(up2)
    bn_up2 = BatchNormalization(name="3dbn_up2")(conv_up2)
    ac_up2 = Activation('relu', name='3dac_up2')(bn_up2)

    up3 = UpSampling3D(size=(2, 2, 2))(ac_up2)
    # up3_sum = add([box[0], up3])
    conv_up3 = Conv3D(96, (3, 3, 3), padding="same", name="3dconv_up3")(up3)
    bn_up3 = BatchNormalization(name="3dbn_up3")(conv_up3)
    ac_up3 = Activation('relu', name='3dac_up3')(bn_up3)

    up4 = UpSampling3D(size=(2, 2, 2))(ac_up3)
    conv_up4 = Conv3D(64, (3, 3, 3), padding="same", name="3dconv_up4")(up4)
    bn_up4 = BatchNormalization(name="3dbn_up4")(conv_up4)
    ac_up4 = Activation('relu', name='3dac_up4')(bn_up4)

    x = Conv3D(3, (1, 1, 1), padding="same", name='3dclassifer')(ac_up4)

    return ac_up4, x



def DenseUNet(img_input, nb_dense_block=4, growth_rate=48, nb_filter=96, reduction=0.0, dropout_rate=0.0, weight_decay=1e-4, classes=1000, weights_path=None):
    '''Instantiate the DenseNet 161 architecture,
        # Arguments
            nb_dense_block: number of dense blocks to add to end
            growth_rate: number of filters to add per dense block
            nb_filter: initial number of filters
            reduction: reduction factor of transition blocks.
            dropout_rate: dropout rate
            weight_decay: weight decay factor
            classes: optional number of classes to classify images
            weights_path: path to pre-trained weights
        # Returns
            A Keras model instance.
    '''
    eps = 1.1e-5
    # compute compression factor
    compression = 1.0 - reduction

    # Handle Dimension Ordering for different backends
    global concat_axis
    concat_axis = 3

    # From architecture for ImageNet (Table 1 in the paper)
    nb_filter = 96
    nb_layers = [6,12,36,24] # For DenseNet-161
    box = []
    # Initial convolution
    x = ZeroPadding2D((3, 3), name='conv1_zeropadding')(img_input)
    x = Conv2D(nb_filter, (7, 7), strides=(2, 2), name='conv1', use_bias=False, trainable=False)(x)
    x = BatchNormalization(epsilon=eps, axis=concat_axis, momentum = 1, name='conv1_bn', trainable=False)(x, training=False)
    x = Scale(axis=concat_axis, name='conv1_scale', trainable=False)(x)
    x = Activation('relu', name='relu1')(x)
    box.append(x)
    x = ZeroPadding2D((1, 1), name='pool1_zeropadding')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        stage = block_idx+2
        x, nb_filter = dense_block(x, stage, nb_layers[block_idx], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)
        box.append(x)
        # Add transition_block
        x = transition_block(x, stage, nb_filter, compression=compression, dropout_rate=dropout_rate, weight_decay=weight_decay)
        nb_filter = int(nb_filter * compression)

    final_stage = stage + 1
    x, nb_filter = dense_block(x, final_stage, nb_layers[-1], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)

    x = BatchNormalization(epsilon=eps, axis=concat_axis, momentum = 1, name='conv'+str(final_stage)+'_blk_bn', trainable=False)(x, training=False)
    x = Scale(axis=concat_axis, name='conv'+str(final_stage)+'_blk_scale', trainable=False)(x)
    x = Activation('relu', name='relu'+str(final_stage)+'_blk')(x)
    box.append(x)

    up0 = UpSampling2D(size=(2,2))(x)
    conv_up0 = Conv2D(768, (3, 3), padding="same", name = "conv_up0", trainable=False)(up0)
    bn_up0 = BatchNormalization(name = "bn_up0", momentum = 1, trainable=False)(conv_up0, training=False)
    ac_up0 = Activation('relu', name='ac_up0')(bn_up0)

    up1 = UpSampling2D(size=(2,2))(ac_up0)
    conv_up1 = Conv2D(384, (3, 3), padding="same", name = "conv_up1", trainable=False)(up1)
    bn_up1 = BatchNormalization(name = "bn_up1", momentum = 1, trainable=False)(conv_up1, training=False)
    ac_up1 = Activation('relu', name='ac_up1')(bn_up1)

    up2 = UpSampling2D(size=(2,2))(ac_up1)
    conv_up2 = Conv2D(96, (3, 3), padding="same", name = "conv_up2", trainable=False)(up2)
    bn_up2 = BatchNormalization(name = "bn_up2", momentum = 1, trainable=False)(conv_up2, training=False)
    ac_up2 = Activation('relu', name='ac_up2')(bn_up2)

    up3 = UpSampling2D(size=(2,2))(ac_up2)
    conv_up3 = Conv2D(96, (3, 3), padding="same", name = "conv_up3", trainable=False)(up3)
    bn_up3 = BatchNormalization(name = "bn_up3", momentum = 1, trainable=False)(conv_up3, training=False)
    ac_up3 = Activation('relu', name='ac_up3')(bn_up3)

    up4 = UpSampling2D(size=(2, 2))(ac_up3)
    conv_up4 = Conv2D(64, (3, 3), padding="same", name="conv_up4", trainable=False)(up4)
    bn_up4 = BatchNormalization(name="bn_up4", momentum = 1, trainable=False)(conv_up4, training=False)
    ac_up4 = Activation('relu', name='ac_up4')(bn_up4)

    x = Conv2D(3, (1,1), padding="same", name='dense167classifer', trainable=False)(ac_up4)

    return ac_up4, x

def conv_block(x, stage, branch, nb_filter, dropout_rate=None, weight_decay=1e-4):
    '''Apply BatchNorm, Relu, bottleneck 1x1 Conv2D, 3x3 Conv2D, and option dropout
        # Arguments
            x: input tensor 
            stage: index for dense block
            branch: layer index within each dense block
            nb_filter: number of filters
            dropout_rate: dropout rate
            weight_decay: weight decay factor
    '''
    eps = 1.1e-5
    conv_name_base = 'conv' + str(stage) + '_' + str(branch)
    relu_name_base = 'relu' + str(stage) + '_' + str(branch)

    # 1x1 Convolution (Bottleneck layer)
    inter_channel = nb_filter * 4
    x = BatchNormalization(epsilon=eps, axis=concat_axis, momentum = 1, name=conv_name_base+'_x1_bn', trainable=False)(x, training=False)
    x = Scale(axis=concat_axis, name=conv_name_base+'_x1_scale', trainable=False)(x)
    x = Activation('relu', name=relu_name_base+'_x1')(x)
    x = Conv2D(inter_channel, (1, 1), name=conv_name_base+'_x1', use_bias=False, trainable=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    # 3x3 Convolution
    x = BatchNormalization(epsilon=eps, axis=concat_axis, momentum = 1, name=conv_name_base+'_x2_bn', trainable=False)(x, training=False)
    x = Scale(axis=concat_axis, name=conv_name_base+'_x2_scale', trainable=False)(x)
    x = Activation('relu', name=relu_name_base+'_x2')(x)
    x = ZeroPadding2D((1, 1), name=conv_name_base+'_x2_zeropadding')(x)
    x = Conv2D(nb_filter, (3, 3), name=conv_name_base+'_x2', use_bias=False, trainable=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    return x


def transition_block(x, stage, nb_filter, compression=1.0, dropout_rate=None, weight_decay=1E-4):
    ''' Apply BatchNorm, 1x1 Convolution, averagePooling, optional compression, dropout 
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_filter: number of filters
            compression: calculated as 1 - reduction. Reduces the number of feature maps in the transition block.
            dropout_rate: dropout rate
            weight_decay: weight decay factor
    '''

    eps = 1.1e-5
    conv_name_base = 'conv' + str(stage) + '_blk'
    relu_name_base = 'relu' + str(stage) + '_blk'
    pool_name_base = 'pool' + str(stage)

    x = BatchNormalization(epsilon=eps, axis=concat_axis, momentum = 1, name=conv_name_base+'_bn', trainable=False)(x, training=False)
    x = Scale(axis=concat_axis, name=conv_name_base+'_scale', trainable=False)(x)
    x = Activation('relu', name=relu_name_base)(x)
    x = Conv2D(int(nb_filter * compression), (1, 1), name=conv_name_base, use_bias=False, trainable=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    x = AveragePooling2D((2, 2), strides=(2, 2), name=pool_name_base)(x)

    return x


def dense_block(x, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1e-4, grow_nb_filters=True):
    ''' Build a dense_block where the output of each conv_block is fed to subsequent ones
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_layers: the number of layers of conv_block to append to the model.
            nb_filter: number of filters
            growth_rate: growth rate
            dropout_rate: dropout rate
            weight_decay: weight decay factor
            grow_nb_filters: flag to decide to allow number of filters to grow
    '''

    eps = 1.1e-5
    concat_feat = x

    for i in range(nb_layers):
        branch = i+1
        x = conv_block(concat_feat, stage, branch, growth_rate, dropout_rate, weight_decay)
        concat_feat = concatenate([concat_feat, x], axis=concat_axis, name='concat_'+str(stage)+'_'+str(branch))

        if grow_nb_filters:
            nb_filter += growth_rate

    return concat_feat, nb_filter
def slice(x, h1, h2):
    """ Define a tensor slice function 
    """
    return x[:, :, :, h1:h2,:]
def slice2d(x, h1, h2):

    tmp = x[h1:h2,:,:,:]
    tmp = tf.transpose(tmp, perm=[1, 2, 0, 3])
    tmp = tf.expand_dims(tmp, 0)
    return tmp

def slice_last(x):

    x = x[:,:,:,:,0]

    return x
def trans(x):

    x = tf.transpose(x, perm=[0,3,1,2,4])

    return x
def trans_back(x):

    x = tf.transpose(x, perm=[0,2,3,1,4])

    return x
def denseunet_3d(args):

    #  ************************3d volume input******************************************************************
    img_input = Input(batch_shape=(args.b, args.input_size, args.input_size, args.input_cols, 1), name='volumetric_data')

    #  ************************(batch*d3cols)*2dvolume--2D DenseNet branch**************************************
    input2d = Lambda(slice, arguments={'h1': 0, 'h2': 2})(img_input)
    single = Lambda(slice, arguments={'h1':0, 'h2':1})(img_input)
    input2d = concatenate([single, input2d], axis=3)
    for i in range(args.input_cols - 2):
        input2d_tmp = Lambda(slice, arguments={'h1': i, 'h2': i + 3})(img_input)
        input2d = concatenate([input2d, input2d_tmp], axis=0)
        if i == args.input_cols - 3:
            final1 = Lambda(slice, arguments={'h1': args.input_cols-2, 'h2': args.input_cols})(img_input)
            final2 = Lambda(slice, arguments={'h1': args.input_cols-1, 'h2': args.input_cols})(img_input)
            final = concatenate([final1, final2], axis=3)
            input2d = concatenate([input2d, final], axis=0)
    input2d = Lambda(slice_last)(input2d)

    #  ******************************stack to 3D volumes *******************************************************
    feature2d, classifer2d = DenseUNet(input2d, reduction=0.5)
    res2d = Lambda(slice2d, arguments={'h1': 0, 'h2': 1})(classifer2d)
    fea2d = Lambda(slice2d, arguments={'h1':0, 'h2':1})(feature2d)
    for j in range(args.input_cols - 1):
        score = Lambda(slice2d, arguments={'h1': j + 1, 'h2': j + 2})(classifer2d)
        fea2d_slice = Lambda(slice2d, arguments={'h1': j + 1, 'h2': j + 2})(feature2d)
        res2d = concatenate([res2d, score], axis=3)
        fea2d = concatenate([fea2d, fea2d_slice], axis=3)

    #  *************************** 3d DenseNet on 3D volume (concate with feature map )*********************************
    res2d_input = Lambda(lambda x: x * 250)(res2d)
    input3d_ori = Lambda(slice, arguments={'h1': 0, 'h2': args.input_cols})(img_input)
    input3d = concatenate([input3d_ori, res2d_input], axis=4)
    feature3d, classifer3d = DenseNet3D(input3d, reduction=0.5)

    final = add([feature3d, fea2d])

    final_conv = Conv3D(64, (3, 3, 3), padding="same", name='fianl_conv')(final)
    final_conv = Dropout(rate=0.1)(final_conv)
    final_bn = BatchNormalization(name="final_bn")(final_conv)
    final_ac = Activation('relu', name='final_ac')(final_bn)
    classifer = Conv3D(3, (1, 1, 1), padding="same", name='2d3dclassifer')(final_ac)


    model = Model( inputs = img_input,outputs = classifer, name='auto3d_residual_conv')

    return model




Overwriting denseunet3d.py


In [4]:
%%writefile loss.py
import keras.backend as K
import tensorflow as tf


def weighted_crossentropy(y_true, y_pred):
    y_pred = y_pred[:,:,:,1:7,:]
    y_true = y_true[:,:,:,1:7,:]
    y_pred_f = K.reshape(y_pred, (-1,3))
    y_true_f = K.reshape(y_true, (-1,))

    soft_pred_f = K.softmax(y_pred_f)
    soft_pred_f = K.log(tf.clip_by_value(soft_pred_f, 1e-10, 1.0))

    neg = K.equal(y_true_f, K.zeros_like(y_true_f))
    neg_calculoss = tf.gather(soft_pred_f[:,0], tf.where(neg))

    pos1 = K.equal(y_true_f, K.ones_like(y_true_f))
    pos1_calculoss = tf.gather(soft_pred_f[:,1], tf.where(pos1))

    pos2 = K.equal(y_true_f, 2*K.ones_like(y_true_f))
    pos2_calculoss = tf.gather(soft_pred_f[:,2], tf.where(pos2))

    loss = -K.mean(tf.concat([0.78*neg_calculoss, 0.65*pos1_calculoss, 8.57*pos2_calculoss], 0))

    return loss

def weighted_crossentropy_2ddense(y_true, y_pred):

    y_pred_f = K.reshape(y_pred, (-1,3))
    y_true_f = K.reshape(y_true, (-1,))

    soft_pred_f = K.softmax(y_pred_f)
    soft_pred_f = K.log(tf.clip_by_value(soft_pred_f, 1e-10, 1.0))

    neg = K.equal(y_true_f, K.zeros_like(y_true_f))
    neg_calculoss = tf.gather(soft_pred_f[:,0], tf.where(neg))

    pos1 = K.equal(y_true_f, K.ones_like(y_true_f))
    pos1_calculoss = tf.gather(soft_pred_f[:,1], tf.where(pos1))

    pos2 = K.equal(y_true_f, 2*K.ones_like(y_true_f))
    pos2_calculoss = tf.gather(soft_pred_f[:,2], tf.where(pos2))

    loss = -K.mean(tf.concat([0.78*neg_calculoss, 0.65*pos1_calculoss, 8.57*pos2_calculoss], 0))

    return loss


Overwriting loss.py


In [5]:
%%writefile train_hybrid.py
from __future__ import print_function
import sys
import torch
from multiprocessing.dummy import Pool as ThreadPool
from medpy.io import load
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from hybridnet import dense_rnn_net
from denseunet3d import denseunet_3d
import keras.backend as K
import os
import time
from loss import weighted_crossentropy
from skimage.transform import resize
import argparse
import os
import numpy as np
import torch
import os
from torch import nn
from torch import optim
from torch.nn import functional as F

class Discriminator(nn.Module):
    def __init__(self, channel=512):
        super(Discriminator, self).__init__()        
        self.channel = channel
        n_class = 1
        
        self.conv1 = nn.Conv3d(1, channel//8, kernel_size=4, stride=2, padding=1)
        self.conv2 = nn.Conv3d(channel//8, channel//4, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm3d(channel//4)
        self.conv3 = nn.Conv3d(channel//4, channel//2, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm3d(channel//2)
        self.conv4 = nn.Conv3d(channel//2, channel, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm3d(channel)
        
        self.conv5 = nn.Conv3d(channel, n_class, kernel_size=4, stride=2, padding=1)
        
    def forward(self, x, _return_activations=False):
        h1 = F.leaky_relu(self.conv1(x), negative_slope=0.2)
        h2 = F.leaky_relu(self.bn2(self.conv2(h1)), negative_slope=0.2)
        h3 = F.leaky_relu(self.bn3(self.conv3(h2)), negative_slope=0.2)
        h4 = F.leaky_relu(self.bn4(self.conv4(h3)), negative_slope=0.2)
        h5 = self.conv5(h4)
        output = h5

        return output
    

class Generator(nn.Module):
    def __init__(self, noise:int=1000, channel:int=64):
        super(Generator, self).__init__()
        _c = channel
        
        self.noise = noise
        self.fc = nn.Linear(1000,512*4*4*4)
        self.bn1 = nn.BatchNorm3d(_c*8)
        
        self.tp_conv2 = nn.Conv3d(_c*8, _c*4, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm3d(_c*4)
        
        self.tp_conv3 = nn.Conv3d(_c*4, _c*2, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn3 = nn.BatchNorm3d(_c*2)
        
        self.tp_conv4 = nn.Conv3d(_c*2, _c, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn4 = nn.BatchNorm3d(_c)
        
        self.tp_conv5 = nn.Conv3d(_c, 1, kernel_size=3, stride=1, padding=1, bias=False)
        
    def forward(self, noise):
        noise = noise.view(-1, 1000)
        h = self.fc(noise)
        h = h.view(-1,512,4,4,4)
        h = F.relu(self.bn1(h))

        h = F.upsample(h,scale_factor = 2)
        h = self.tp_conv2(h)
        h = F.relu(self.bn2(h))
        
        h = F.upsample(h,scale_factor = 2)
        h = self.tp_conv3(h)
        h = F.relu(self.bn3(h))

        h = F.upsample(h,scale_factor = 2)
        h = self.tp_conv4(h)
        h = F.relu(self.bn4(h))

        h = F.upsample(h,scale_factor = 2)
        h = self.tp_conv5(h)

        h = F.tanh(h)

        return h
try:
  os.mkdir('/content/Experiments')
except:
  pass
parser = argparse.ArgumentParser(description='Keras DenseUnet Training')
parser.add_argument('-data', type=str, default='data/', help='test images')
parser.add_argument('-save_path', type=str, default='/content/Experiments/')
parser.add_argument('-b', type=int, default=1)
parser.add_argument('-input_size', type=int, default=224)
parser.add_argument('-model_weight', type=str, default='./model/model_best.hdf5')
parser.add_argument('-input_cols', type=int, default=8)
parser.add_argument('-arch', type=str, default='')
parser.add_argument('-mean', type=int, default=48)
args = parser.parse_args()
thread_num = 14
liverlist = [32,34,38,41,47,87,89,91,105,106,114,115,119]
def load_seq_crop_data_masktumor_try(Parameter_List):
    img = Parameter_List[0]
    tumor = Parameter_List[1]
    lines = Parameter_List[2]
    numid = Parameter_List[3]
    minindex = Parameter_List[4]
    maxindex = Parameter_List[5]
    #  randomly scale
    scale = np.random.uniform(0.8,1.2)
    deps = int(args.input_size * scale)
    rows = int(args.input_size * scale)
    cols = args.input_cols

    sed = np.random.randint(1,numid)
    cen = lines[sed-1]
    cen = np.fromstring(cen, dtype=int, sep=' ')
    # print (cen)
    a = min(max(minindex[0] + deps/2, cen[0]), maxindex[0]- deps/2-1)
    b = min(max(minindex[1] + rows/2, cen[1]), maxindex[1]- rows/2-1)
    c = min(max(minindex[2] + cols/2, cen[2]), maxindex[2]- cols/2-1)

    if a < deps / 2:
      a1=int((img.shape[0]/2)-(deps/2))
      a2=int((img.shape[0]/2)+(deps/2))
    else:
      a1=int(a-deps/2)
      a2=int(a+deps/2)
    if b < rows / 2:
      b1=int((img.shape[0]/2)-(rows/2))
      b2=int((img.shape[0]/2)+(rows/2))
    else:
      b1=int(b-rows/2)
      b2=int(b+rows/2)
    if c < cols / 2:
      c1=int((img.shape[0]/2)-(cols/2))
      c2=int((img.shape[0]/2)+(cols/2))
    else:
      c1=int(c-cols/2)
      c2=int(c+cols/2)

    cropp_img = img[a1:a2, b1:b2,c1: c2+1].copy()

    cropp_tumor = tumor[a1:a2, b1:b2,c1: c2+1].copy()
    

    cropp_img -= args.mean
    flip_num = np.random.randint(0,8)
    if flip_num == 1:
      cropp_img = np.flipud(cropp_img)
      cropp_tumor = np.flipud(cropp_tumor)
    elif flip_num == 2:
      cropp_img = np.fliplr(cropp_img)
      cropp_tumor = np.fliplr(cropp_tumor)
    elif flip_num == 3:
      cropp_img = np.rot90(cropp_img, k=1, axes=(1, 0))
      cropp_tumor = np.rot90(cropp_tumor, k=1, axes=(1, 0))
    elif flip_num == 4:
      cropp_img = np.rot90(cropp_img, k=3, axes=(1, 0))
      cropp_tumor = np.rot90(cropp_tumor, k=3, axes=(1, 0))
    elif flip_num == 5:
      cropp_img = np.fliplr(cropp_img)
      cropp_tumor = np.fliplr(cropp_tumor)
      cropp_img = np.rot90(cropp_img, k=1, axes=(1, 0))
      cropp_tumor = np.rot90(cropp_tumor, k=1, axes=(1, 0))
    elif flip_num == 6:
      cropp_img = np.fliplr(cropp_img)
      cropp_tumor = np.fliplr(cropp_tumor)
      cropp_img = np.rot90(cropp_img, k=3, axes=(1, 0))
      cropp_tumor = np.rot90(cropp_tumor, k=3, axes=(1, 0))
    elif flip_num == 7:
      cropp_img = np.flipud(cropp_img)
      cropp_tumor = np.flipud(cropp_tumor)
      cropp_img = np.fliplr(cropp_img)
      cropp_tumor = np.fliplr(cropp_tumor)
    cropp_tumor = resize(cropp_tumor, (args.input_size,args.input_size, args.input_cols), order=0, mode='edge', cval=0, clip=True, preserve_range=True)
    cropp_img   = resize(cropp_img, (args.input_size,args.input_size, args.input_cols), order=3, mode='constant', cval=0, clip=True, preserve_range=True)
    return cropp_img, cropp_tumor
   


def generate_arrays_from_file(batch_size, trainidx, img_list, tumor_list, tumorlines, liverlines, tumoridx, liveridx, minindex_list, maxindex_list,gen):
    while 1:
      choice=np.random.rand()
      # choice=0
      if choice<0.5:
        X = np.zeros((batch_size, args.input_size, args.input_size, args.input_cols,1), dtype='float32')
        Y = np.zeros((batch_size, args.input_size, args.input_size, args.input_cols,1), dtype='int16')
        Parameter_List = []
        for idx in range(batch_size):
            count = np.random.choice(trainidx)
            img = img_list[count]
            tumor = tumor_list[count]
            minindex = minindex_list[count]
            maxindex = maxindex_list[count]
            num = np.random.randint(0,6)
            if num < 3 or (count in liverlist):
                lines = liverlines[count]
                numid = liveridx[count]
            else:
                lines = tumorlines[count]
                numid = tumoridx[count]
            Parameter_List.append([img, tumor, lines, numid, minindex, maxindex])
        pool = ThreadPool(thread_num)
        result_list = pool.map(load_seq_crop_data_masktumor_try, Parameter_List)
        pool.close()
        pool.join()
        for idx in range(len(result_list)):
            X[idx, :, :, :, 0] = result_list[idx][0]
            Y[idx, :, :, :, 0] = result_list[idx][1]
        if np.sum(Y==0)==0:
            continue
        if np.sum(Y==1)==0:
            continue
        if np.sum(Y==2)==0:
            continue
      else:
        print('gan')
        noise = torch.randn((1,1000, 1, 1 ,1))
        a=gen(noise).detach().numpy()
        a=resize(a[0,0], (1,224,224,16,1), order=0, mode='edge', cval=0, clip=True, preserve_range=True)
        
        X,Y=a[:,:,:,:8,:],a[:,:,:,8:,:]
      yield (X,Y) 
def train_and_predict(args):
    
    device = torch.device( 'cuda')
    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)

    if args.arch == "3dpart":
        model = denseunet_3d(args).cuda()
        model_path = "3dpart_model"
        sgd = SGD(lr=1e-5, momentum=0.9, nesterov=True)
        model.compile(optimizer=sgd, loss=[weighted_crossentropy])

        model1 = denseunet_3d(args).cuda()
        sgd1 = SGD(lr=1e-5, momentum=0.9, nesterov=True)
        model1.compile(optimizer=sgd1, loss=[weighted_crossentropy])
        for en,i in enumerate(list(model1.layers)):
          if en<1000:
            i.requires_grad=False


        model2 = denseunet_3d(args).cuda()
        sgd2 = SGD(lr=1e-5, momentum=0.9, nesterov=True)
        model2.compile(optimizer=sgd2, loss=[weighted_crossentropy])
        for en,i in enumerate(list(model2.layers)):
          if en<1000:
            i.requires_grad=False


        # model3 = denseunet_3d(args)
        # sgd3 = SGD(lr=1e-3, momentum=0.9, nesterov=True)
        # model3.compile(optimizer=sgd3, loss=[weighted_crossentropy])


        # model4 = denseunet_3d(args)
        # sgd4 = SGD(lr=1e-3, momentum=0.9, nesterov=True)
        # model4.compile(optimizer=sgd4, loss=[weighted_crossentropy])

        models=[model1,model2]
        opts=[sgd1,sgd2]

    else:
        model = dense_rnn_net(args)
        model_path = "hybrid_model"
        sgd = SGD(lr=1e-3, momentum=0.9, nesterov=True)
        model.compile(optimizer=sgd, loss=[weighted_crossentropy])


        model1 = dense_rnn_net(args)
        sgd1 = SGD(lr=1e-3, momentum=0.9, nesterov=True)
        model1.compile(optimizer=sgd1, loss=[weighted_crossentropy])


        model2 = dense_rnn_net(args)
        sgd2 = SGD(lr=1e-3, momentum=0.9, nesterov=True)
        model2.compile(optimizer=sgd2, loss=[weighted_crossentropy])


        # model3 = dense_rnn_net(args)
        # sgd3 = SGD(lr=1e-3, momentum=0.9, nesterov=True)
        # model3.compile(optimizer=sgd3, loss=[weighted_crossentropy])


        # model4 = dense_rnn_net(args)
        # sgd4 = SGD(lr=1e-3, momentum=0.9, nesterov=True)
        # model4.compile(optimizer=sgd4, loss=[weighted_crossentropy])

        models=[model1,model2]
        opts=[sgd1,sgd2]

    trainidx = list(range(2))
    img_list = []
    tumor_list = []
    minindex_list = []
    maxindex_list = []
    tumorlines = []
    tumoridx = []
    liveridx = []
    liverlines = []
    for idx in range(2):
        img, img_header = load('/content/data/myTrainingData/volume-'+str(idx)+'.nii')
        tumor, tumor_header = load('/content/data/myTrainingData/segmentation-'+str(idx)+'.nii')
        img_list.append(img)
        tumor_list.append(tumor)

        maxmin = np.loadtxt('/content/data/myTrainingDataTxt/LiverBox/box_'+str(idx)+'.txt')
        minindex = maxmin[0:3]
        maxindex = maxmin[3:6]
        minindex = np.array(minindex, dtype='int')
        maxindex = np.array(maxindex, dtype='int')
        minindex[0] = max(minindex[0]-3, 0)
        minindex[1] = max(minindex[1]-3, 0)
        minindex[2] = max(minindex[2]-3, 0)
        maxindex[0] = min(img.shape[0], maxindex[0]+3)
        maxindex[1] = min(img.shape[1], maxindex[1]+3)
        maxindex[2] = min(img.shape[2], maxindex[2]+3)
        minindex_list.append(minindex)
        maxindex_list.append(maxindex)

        f1 = open('/content/data/myTrainingDataTxt/TumorPixels/tumor_'+str(idx)+'.txt')
        tumorline = f1.readlines()
        tumorlines.append(tumorline)
        tumoridx.append(len(tumorline))
        f1.close()

        f2 = open('/content/data/myTrainingDataTxt/LiverPixels/liver_'+str(idx)+'.txt')
        liverline = f2.readlines()
        liverlines.append(liverline)
        liveridx.append(len(liverline))
        f2.close()
    
    if not os.path.exists(args.save_path +model_path):
        os.mkdir(args.save_path + model_path)
    if not os.path.exists(args.save_path + "history"):
        os.mkdir(args.save_path + 'history')
    else:
        if os.path.exists(args.save_path + "history/lossbatch.txt"):
            os.remove(args.save_path + 'history/lossbatch.txt')
        if os.path.exists(args.save_path + "history/lossepoch.txt"):
            os.remove(args.save_path + 'history/lossepoch.txt')
    print('-'*30)
    print('Fitting model......')
    print('-'*30)

    #change
    #no checkpoint
    # steps = 27386 / (args.b * 6)
    steps = 5
    gen=Generator()
    a=torch.load('/content/gdrive/MyDrive/generator_99.pth')
    gen.load_state_dict(a['model_state_dict'])
    for epoch in range(5):
      for i in range(2):
        models[i].fit_generator(generate_arrays_from_file(args.b, list(range(1)), [img_list[i]], [tumor_list[i]], [tumorlines[i]], [liverlines[i]],
                                                  [tumoridx[i]], [liveridx[i]], [minindex_list[i]], [maxindex_list[i]],gen),
                        steps_per_epoch=5,
                        epochs= 1, verbose = 1, max_queue_size=10, workers=3, use_multiprocessing=True)
      
      for num in tqdm(range(1000,len(model.layers))):
          wts=model.layers[num].get_weights()
          if len(wts)!=0:
            ls=[]
            for i in range(len(wts)):
              temp = np.zeros(wts[i].shape)
              for s in range(2):
                weights=models[s].layers[num].get_weights()[i]
                noise = np.random.normal(0, 1e-5*np.std(weights),weights.shape)
                # noise = torch.from_numpy(noise).float().to('cuda')
                temp += 0.5*(weights+noise)
              ls.append(tf.convert_to_tensor(temp)._copy_to_device('/device:GPU:0'))
            model.layers[num].set_weights(ls)
            for s in range(2):
              models[s].layers[num].set_weights(ls)
      # models[0].cuda()
      # models[1].cuda()
      # model.cuda()
      


if __name__ == '__main__':
    train_and_predict(args)

Overwriting train_hybrid.py


In [ ]:
!CUDA_VISIBLE_DEVICES='0' python train_hybrid.py -arch end2end

2021-06-05 18:14:23.625511: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
------------------------------
Creating and compiling model...
------------------------------
2021-06-05 18:14:25.601521: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-06-05 18:14:25.631220: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-06-05 18:14:25.631827: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2021-06-05 18:14:25.631870: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened d